In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
# OpenAIEmbeddings is used to generate vector embeddings for text using OpenAI models.
# Chroma is used as a vector store to efficiently store and search embeddings.
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
import pandas as pd

# Load the text file
books = pd.read_csv("cleaned_books.csv")

In [9]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep="\n",
                                   index=False, 
                                   header=False)

In [10]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()

'''
CharacterTextSplitter :- This text splitter is used to split the text into chunks based on a specified character length.
    chunk_size: The maximum size of each chunk in characters.
    chunk_overlap: The number of characters that should overlap between chunks.
    separator: The character used to split the text into chunks, default is newline.

'''
text_splitter = CharacterTextSplitter(chunk_size = 0, chunk_overlap = 0, separator = "\n")

documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

In [11]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [15]:
from typing import List
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
from langchain_core.documents import Document # Import Document class if using it in type hints

import os

class CustomEmbeddings(Embeddings):
    def __init__(self, model_name: str, batch_size: int = 32):
        self.model = SentenceTransformer(model_name)
        self.batch_size = batch_size

    # The 'documents' parameter here will be a list of LangChain Document objects
    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        """
        Embed a list of Document objects by extracting their page_content.
        """
        # Extract the string content from each Document object
        # texts_to_embed = [doc.page_content for doc in documents]
        
        # Now pass the list of strings to the SentenceTransformer model
        embeddings = self.model.encode(documents, convert_to_numpy=True, batch_size=self.batch_size)
        return embeddings.tolist()

    def embed_query(self, query: str) -> List[float]:
        return self.model.encode([query])[0].tolist()

    

'''
https://github.com/langchain-ai/langchain/discussions/7818
'''
# --- Main Code Execution ---

# 1. Instantiate your custom embedding model
# Faster for testing/smaller models
model_name = "sentence-transformers/all-MiniLM-L6-v2"

# You can adjust the batch_size here
embedding_model = CustomEmbeddings(model_name=model_name, batch_size=64)

print(f"Embedding model '{model_name}' loaded successfully with batch size {embedding_model.batch_size}.")

# 2. Create or Load the Chroma vector database
persist_directory = "./chroma_db"

# Check if the database already exists
if os.path.exists(persist_directory) and os.listdir(persist_directory):
    print(f"Loading existing Chroma database from '{persist_directory}'...")
    db_books = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)
    print("Chroma database loaded.")
else:
    print(f"Creating new Chroma database and embedding documents to '{persist_directory}'...")
    db_books = Chroma.from_documents(
        documents,
        embedding_model,
        persist_directory=persist_directory # Persist the database to disk
    )
    print("Chroma database created and documents embedded.")




Embedding model 'sentence-transformers/all-MiniLM-L6-v2' loaded successfully with batch size 64.
Loading existing Chroma database from './chroma_db'...
Chroma database loaded.


In [17]:

# 3. Perform a similarity search (demonstration)
query = "A book to teach children about nature"
print(f"\nSearching for documents similar to: '{query}'")

docs = db_books.similarity_search(query, k=10)
docs


Searching for documents similar to: 'A book to teach children about nature'


[Document(id='3da26738-d210-4d6d-867a-73fc02b7db45', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='17d7fafd-0c9b-47e2-abe5-6aaed0c9a791', metadata={'source': 'tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='a3b7dccb-b15a-4edb-885e-9d68b63ce1a9', metadata={'source': '

In [ ]:
# This line finds the row(s) in the 'books' DataFrame where the 'isbn13' matches the ISBN extracted from the most similar document.
books[books["isbn13"] == int(docs[0].page_content.split(" ")[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [22]:

def retrieve_sementic_recommentdation(
    query: str,
    top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=top_k)
    # recs_df = pd.DataFrame([doc.page_content for doc in recs], columns
    
    books_list = []
    for doc in recs:
        books_list += [int(doc.page_content.strip('"').split()[0])]
        
    return books[books["isbn13"].isin(books_list)].head(top_k)
    
    

In [24]:
retrieve_sementic_recommentdation("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit..."
406,9780064403870,0064403874,"R-T, Margaret, and the Rats of NIMH",Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=WTHHH...,"When Margaret and her younger brother, Artie, ...",1991.0,3.52,272.0,631.0,"R-T, Margaret, and the Rats of NIMH",9780064403870 When Margaret and her younger br...
1642,9780374522599,0374522596,The Control of Nature,John McPhee,Nature,http://books.google.com/books/content?id=p1qKQ...,The Control of Nature is John McPhee's bestsel...,1990.0,4.24,288.0,3365.0,The Control of Nature,9780374522599 The Control of Nature is John Mc...
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...
3748,9780786808373,0786808373,Baby Einstein: Birds,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=0jxHP...,"Introducing your baby to birds, cats, dogs, an...",2002.0,3.78,20.0,9.0,Baby Einstein: Birds,"9780786808373 Introducing your baby to birds, ..."
3749,9780786808380,0786808381,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=jv4NA...,"Introduce your babies to birds, cats, dogs, an...",2002.0,4.03,20.0,29.0,Baby Einstein: Babies,"9780786808380 Introduce your babies to birds, ..."
3750,9780786808397,078680839X,Baby Einstein: Dogs,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=qut8t...,"Introduce your baby to birds, cats, dogs, and ...",2002.0,3.81,20.0,26.0,Baby Einstein: Dogs,"9780786808397 Introduce your baby to birds, ca..."
3765,9780786819119,0786819111,"Baby Einstein: Water, Water Everywhere","Disney Book Group,",Juvenile Fiction,http://books.google.com/books/content?id=tuAdA...,Charming illustrations and playful rhythmic ve...,2003.0,3.70,10.0,77.0,"Baby Einstein: Water, Water Everywhere",9780786819119 Charming illustrations and playf...
4898,9781593851170,1593851170,The Nature of Play,Anthony D. Pellegrini;Peter K. Smith,Psychology,http://books.google.com/books/content?id=Nukz6...,"""Comprehensive and up to date, this tightly ed...",2005.0,4.25,308.0,4.0,The Nature of Play: Great Apes and Humans,"9781593851170 ""Comprehensive and up to date, t..."
